<a href="https://colab.research.google.com/github/Shreyas-13/Tensorflow-Developer/blob/main/00_Tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Fundamentals

Outline:
* Intro to tensors
* Getting info from tensors
* Manipulating tensors
* Tensors and Numpy
* Using @tf.function(a way to speed up regular python func)
* Using GPU with tensorflow

# Intro to Tensors

### Creating tensors using `tf.constant`

In [1]:
import tensorflow as tf

print(tf.__version__)

##Creating a tensor
scalar = tf.constant(7)
scalar

2.8.0


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [2]:
#Creating a vector

vector = tf.constant([13,2], dtype=tf.float16)
vector

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([13.,  2.], dtype=float16)>

In [3]:
vector.ndim

1

In [4]:
#Creating a matrix

matrix = tf.constant([[10,20],
                      [20,30],
                     [30,40]])
matrix

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10, 20],
       [20, 30],
       [30, 40]], dtype=int32)>

In [5]:
matrix.ndim

2

In [6]:
#Creating a 3-D tensor

tensor = tf.constant([[[10,20,30],
                       [20,30,40]],
                      [[13,2,2000],
                       [13,2,2001]],
                      [[16,11,2002],
                       [12,12,12]],
                      [[12,23,43],
                      [12,45,76]]])
tensor

<tf.Tensor: shape=(4, 2, 3), dtype=int32, numpy=
array([[[  10,   20,   30],
        [  20,   30,   40]],

       [[  13,    2, 2000],
        [  13,    2, 2001]],

       [[  16,   11, 2002],
        [  12,   12,   12]],

       [[  12,   23,   43],
        [  12,   45,   76]]], dtype=int32)>

In [7]:
tensor.ndim

3

* scalar: A variable with single value
* vector: A set of values in single dimension[a single subscript is requiured hence 1-D]
* matrix: A set of vector having 2-dimension[2 subscripts required hence 2-D e.g. arr[0][0]]
* tensor: A set of matrices having n-dimension(dimension refers to the no. of digits required to indicate a particular element in an array)

### Creating tensors with `tf.Variable`

In [8]:
##Creating a tensor using tf.Variable

changable_tensor = tf.Variable([10,23])
changable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10, 23], dtype=int32)>

In [9]:
##Updating the tensor value
changable_tensor[0].assign(13)
changable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([13, 23], dtype=int32)>

Summary:
  * tf.Variable unlike tf.constant can create tensors that can be updated when required

**Note-** It is good practice to declare your tensors as constant and change them to variable when required.

### Creating Random Tensors

Random tensors are used when we're trying to initialize the weights of a NN

In [10]:
'''Creating a random tensor with with values ranging b/w -1,1 i.e. Normal
Distribution'''
tf.random.set_seed(1)

random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

random_1 == random_2

'''We can also use Uniform Distribution if we want a value between 0,1'''

random_3 = tf.random.Generator.from_seed(9)
random_3 = random_3.uniform(shape=(1,2))
random_3

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.8203654 , 0.70470357]], dtype=float32)>

### Shuffling the tensors

Required so that the dataset is mixedd randomly and not same type of data is repeated together.

In [11]:
import numpy as np

matrix = np.random.rand(6).reshape(3,2)

not_shuffeled = tf.constant(matrix)
not_shuffeled

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.56946872, 0.11395717],
       [0.07649994, 0.50488376],
       [0.56584344, 0.85768969]])>

In [12]:
'''Here even using seed doesn't guarantee same output everytime cz of global and 
operational-level seed'''

shuffeled = tf.random.shuffle(not_shuffeled, seed=1)
shuffeled

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.56946872, 0.11395717],
       [0.07649994, 0.50488376],
       [0.56584344, 0.85768969]])>

In [13]:
'''Here setting tf.random.set_seed() resolves the problem'''

tf.random.set_seed(1)
shuffeled = tf.random.shuffle(not_shuffeled, seed=11)
shuffeled

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.07649994, 0.50488376],
       [0.56584344, 0.85768969],
       [0.56946872, 0.11395717]])>

### Getting Information from tensors

A tensor has following attributes:
  * Shape - Shape of tensor tensor.shape
  * Rank - No of dimensions tensor.ndim
  * Axis - tensor[0] or tensor[:, 1]
  * Size - No of elements in the tensor tf.size(tensor)

In [18]:
##func to extract all these info

def info(tensor):
  print('Datatype of Elements: ', tensor.dtype)
  print('Rank: ', tensor.ndim)
  print('Shape: ', tensor.shape)
  print('Axis 0: ', tensor[0])
  print('Size: ', tf.size(tensor).numpy())

if __name__ == '__main__':
  tf.random.set_seed(100)
  rank_4_tensor = tf.random.Generator.from_seed(32)
  rank_4_tensor = rank_4_tensor.normal(shape=(2,3,4,5))
  info(rank_4_tensor)

Datatype of Elements:  <dtype: 'float32'>
Rank:  4
Shape:  (2, 3, 4, 5)
Axis 0:  tf.Tensor(
[[[ 0.7901182   1.585549    0.4356279   0.23645182 -0.1589871 ]
  [ 1.302304    0.9592239   0.85874265 -1.5181769   1.4020647 ]
  [ 1.5570306  -0.96762174  0.495291   -0.648484   -1.8700892 ]
  [ 2.7830641  -0.645002    0.18022095 -0.14656258  0.34374258]]

 [[ 0.41367555  0.17573498 -1.0871261   0.45905176  0.20386009]
  [ 0.562024   -2.3001142  -1.349454    0.81485     1.2790666 ]
  [ 0.02203509  1.5428121   0.78953624  0.53897345 -0.48535708]
  [ 0.74055266  0.31662667 -1.4391748   0.58923835 -1.4268045 ]]

 [[-0.7565803  -0.06854702  0.07595026 -1.2573844  -0.23193763]
  [-1.8107855   0.09988727 -0.50998646 -0.7535805  -0.57166284]
  [ 0.1480774  -0.23362993 -0.3522796   0.40621263 -1.0523509 ]
  [ 1.2054597   1.6874489  -0.4462975  -2.3410842   0.99009085]]], shape=(3, 4, 5), dtype=float32)
Size:  120


### Indexing Tensor